# First steps: a synthetic data study

In this part of the tutorial, we'll take a [myokit simulation](https://myokit.readthedocs.io/api_simulations/Simulation.html) and wrap it in a [PINTS ForwardModel](https://pints.readthedocs.io/en/latest/core_classes_and_methods.html#forward-model).
We'll then use this statistical model to define an optimisation problem and attempt to solve it.







### Intermezzo: Model?

The word "model" gets a lot of mileage in computational electrophysiology.
This can get a bit confusing, some of the main uses in this tutorial are shown again here:

- An expression system, or even an isolated myocyte, acts as a biological model for the true system of interest: myocytes in the intact human heart.
- A model of an ion current is a set of mathematical equations describing how the state of the underlying ion channels changes in response to a voltage signal. This is the sense in which Myokit uses the word "model".
- A model in PINTS is something you throw parameters at, that then returns a signal. This is a parametrisable simulation in Myokit.
- A noise model describes how the output of a measurement is affected by different noise sources (e.g. anything that isn't the system of interest).

